<a href="https://colab.research.google.com/github/tomtomh512/UBH24/blob/main/UBH24_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install opendatasets

In [81]:
import opendatasets as od
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import pair_confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [82]:
od.download("https://www.kaggle.com/datasets/rm1000/skin-cancer-isic-images")

Skipping, found downloaded files in "./skin-cancer-isic-images" (use force=True to force download)


In [83]:
transform = transforms.Compose([
  transforms.Resize((128, 128)),          # Resize to a 128x128
  transforms.Grayscale(),                 # Convert to grayscale
  transforms.ToTensor(),                  # Convert to tensor
  transforms.Normalize([0.5], [0.5])      # Normalize
])

In [84]:
train_data = datasets.ImageFolder(root='skin-cancer-isic-images', transform=transform)
test_data = datasets.ImageFolder(root='skin-cancer-isic-images', transform=transform)
train_loader = DataLoader(train_data, batch_size = 30, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 30, shuffle = False)

In [85]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)       # input, output, kernel, stride
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.conv3 = nn.Conv2d(64, 128, 3, 1)
    self.conv4 = nn.Conv2d(128, 256, 3, 1)

    self.fc1 = nn.Linear(256 * 6 * 6, 512)    # 256 * 6 * 6 -> 512
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 128)
    self.fc4 = nn.Linear(128, 2)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2)
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)
    X = F.relu(self.conv3(X))
    X = F.max_pool2d(X, 2, 2)
    X = F.relu(self.conv4(X))
    X = F.max_pool2d(X, 2, 2)

    X = X.view(-1, 256 * 6 * 6)

    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = F.relu(self.fc3(X))
    X = self.fc4(X)

    return F.log_softmax(X, dim = 1)

In [86]:
model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [87]:
epochs = 10

model.train()
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    images, labels = data

    optimizer.zero_grad()

    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * images.size(0)

  epoch_loss = running_loss / len(train_loader.dataset)
  print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.3f}')

print('Finished Training')

Epoch 1/10, Loss: 0.686
Epoch 2/10, Loss: 0.631
Epoch 3/10, Loss: 0.586
Epoch 4/10, Loss: 0.549
Epoch 5/10, Loss: 0.507
Epoch 6/10, Loss: 0.453
Epoch 7/10, Loss: 0.418
Epoch 8/10, Loss: 0.370
Epoch 9/10, Loss: 0.353
Epoch 10/10, Loss: 0.324
Finished Training


In [88]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for i, data in enumerate(test_loader, 0):
    images, labels = data

    outputs = model(images)
    predicted = outputs.argmax(dim=1)
    total += labels.size(0)

    # Count number of matches
    correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f'Accuracy: {accuracy:.3f}%')

Accuracy: 88.474%


In [89]:
torch.save(model.state_dict(), 'model_0.pt')

In [91]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('L')     # 'L' = Single channel image, grayscale
    transform = transforms.Compose([                # Match transform composition
        transforms.Resize((128, 128)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    image = transform(image)
    return image

In [99]:
image_path = 'skin-cancer-isic-images/malignant/0056.jpg'
processed_image = preprocess_image(image_path)
processed_image = processed_image.unsqueeze(0)  # Add batch dimension
model.eval()
with torch.no_grad():
    new_prediction = model(processed_image)

print(new_prediction.argmax())

tensor(1)
